In [54]:
import numpy as np
import pandas as pd
from graphtools import *
import networkx as nx
from tqdm import tqdm

In [16]:
df_feature_def = pd.read_csv('./Group_Project/feature_definitions.csv')
df_feature_def[df_feature_def['Variable']=='opencred_647L']

,Variable,Description
335,opencred_647L,Number of active loans from the previous appli...


In [17]:
# selects only depth=0 train tables

# base
df_train_base = pd.read_csv('./Group_Project/train_base.csv/train_base.csv')

# statics
df_train_static_0_0 = pd.read_csv('./Group_Project/train_static_0_0.csv/train_static_0_0.csv')
df_train_static_0_1 = pd.read_csv('./Group_Project/train_static_0_1.csv/train_static_0_1.csv')

# statics cb
df_train_static_cb_0 = pd.read_csv('./Group_Project/train_static_cb_0.csv/train_static_cb_0.csv')


C:\Users\Oshadha Senanayake\AppData\Local\Temp\ipykernel_18736\3263998238.py:7: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_static_0_0 = pd.read_csv('./Group_Project/train_static_0_0.csv/train_static_0_0.csv')
C:\Users\Oshadha Senanayake\AppData\Local\Temp\ipykernel_18736\3263998238.py:8: DtypeWarning: Columns (20,45,46,56,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_static_0_1 = pd.read_csv('./Group_Project/train_static_0_1.csv/train_static_0_1.csv')
C:\Users\Oshadha Senanayake\AppData\Local\Temp\ipykernel_18736\3263998238.py:11: DtypeWarning: Columns (1,2,3,4,7,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_static_cb_0 = pd.read_csv('./Group_Project/train_static_cb_0.csv/train_static_cb_0.csv')


In [18]:
print(df_train_base.shape)
print(df_train_base.head(2))

(1526659, 5)
   case_id date_decision   MONTH  WEEK_NUM  target
0        0    2019-01-03  201901         0       0
1        1    2019-01-03  201901         0       0


In [19]:
print(df_train_static_0_1.shape)
print(df_train_static_0_0.shape)

(522902, 168)
(1003757, 168)


In [21]:
# concat train static tables
df_train_static = pd.concat([df_train_static_0_0,df_train_static_0_1])

# merge basetable with static table
df = pd.merge(df_train_base,df_train_static,how='left',on='case_id')

# create final dataframe
df = pd.merge(df,df_train_static_cb_0,how='left',on='case_id')

In [22]:
sel_non_imputated_cols = ['actualdpdtolerance_344P','amtinstpaidbefduel24m_4187115A','annuity_780A','annuitynextmonth_57A','applicationcnt_361L','applications30d_658L','applicationscnt_1086L','applicationscnt_464L','applicationscnt_629L','applicationscnt_867L','avgdbddpdlast24m_3658932P','avgdbddpdlast3m_4187120P','avgdbdtollast24m_4525197P','avgdpdtolclosure24_3658938P','avginstallast24m_3658937A','avgmaxdpdlast9m_3716943P','avgoutstandbalancel6m_4187114A','avgpmtlast12m_4525200A','clientscnt_100L','clientscnt_1022L','clientscnt_1071L','clientscnt_1130L','clientscnt_157L','clientscnt_257L','clientscnt_304L','clientscnt_360L','clientscnt_493L','clientscnt_533L','clientscnt_887L','clientscnt_946L','clientscnt12m_3712952L','clientscnt3m_3712950L','clientscnt6m_3712949L','credamount_770A','credtype_322L','currdebt_22A','currdebtcredtyperange_828A','daysoverduetolerancedd_3976961L','deferredmnthsnum_166L','disbursedcredamount_1113A','disbursementtype_67L','downpmt_116A','isbidproduct_1095L','lastapprcommoditycat_1041M','lastapprcommoditytypec_5251766M','lastcancelreason_561M','lastrejectcommoditycat_161M','lastrejectcommodtypec_5251769M','lastrejectreason_759M','lastrejectreasonclient_4145040M','mobilephncnt_593L','numactivecreds_622L','numactivecredschannel_414L','numactiverelcontr_750L','numcontrs3months_479L','numnotactivated_1143L','numpmtchanneldd_318L','numrejects9m_859L','previouscontdistrict_112M','sellerplacecnt_915L','sellerplacescnt_216L']
sel_imputated_cols = ['bankacctype_710L','cardtype_51L','cntincpaycont9m_3716944L','cntpmts24_3658933L','commnoinclast6m_3546845L','eir_270L','inittransactioncode_186L','interestrate_311L','lastapprcredamount_781A','lastrejectcredamount_222A','lastst_736L','maininc_215A','maxannuity_159A','maxdbddpdtollast6m_4187119P','numinstls_657L','numinstlsallpaid_934L','numinstpaidearly_338L','numinstregularpaid_973L','numinsttopaygr_769L','numinstunpaidmax_3546851L','opencred_647L','pmtnum_254L','posfpd30lastmonth_3976960P','price_1097A','sumoutstandtotal_3546847A','totaldebt_9A','totalsettled_863A']
imputate_with_zero_cols = ['cntincpaycont9m_3716944L','cntpmts24_3658933L','commnoinclast6m_3546845L','lastapprcredamount_781A','lastrejectcredamount_222A','maininc_215A','maxannuity_159A','numinstls_657L','numinstlsallpaid_934L','numinstpaidearly_338L','numinstregularpaid_973L','numinsttopaygr_769L','numinstunpaidmax_3546851L','pmtnum_254L','posfpd30lastmonth_3976960P','price_1097A','sumoutstandtotal_3546847A','totaldebt_9A','totalsettled_863A']

In [67]:
# Calculate the percentage of null values in each column
null_percentage = df.isnull().mean() * 100
# Create a new dataframe with column names and their respective null percentages
result_df = pd.DataFrame({'Column Name': null_percentage.index, 'Null Percentage': null_percentage.values})
result_df

,Column Name,Null Percentage
0,case_id,0.000000
1,date_decision,0.000000
2,MONTH,0.000000
3,WEEK_NUM,0.000000
4,target,0.000000
...,...,...
219,responsedate_4917613D,85.267699
220,riskassesment_302T,96.491751
221,riskassesment_940T,96.491685
222,secondquarter_766L,9.233758


In [23]:
def impute_with_zero(column):
    df[column] = df[column].fillna(0.0)
    
# imputation with zero
impute_with_zero(imputate_with_zero_cols)

In [24]:
# manual imputation

# bankacctype_710L
df['bankacctype_710L'] = df['bankacctype_710L'].fillna('NA')

# cardtype_51L
df['cardtype_51L'] = df['cardtype_51L'].fillna('NOCARD')

# eir_270L
df['eir_270L'] = df['eir_270L'].fillna(0.2)

# inittransactioncode_186L
df['inittransactioncode_186L'] = df['inittransactioncode_186L'].fillna(df['inittransactioncode_186L'].mode()[0])

# interestrate_311L
df['interestrate_311L'] = df['interestrate_311L'].fillna(df['interestrate_311L'].mean())

# lastst_736L
df['lastst_736L'] = df['lastst_736L'].fillna(df['lastst_736L'].mode()[0])

# opencred_647L
df['opencred_647L'] = df['opencred_647L'].fillna(df['opencred_647L'].mode()[0])

In [25]:
# selected columns for the final dataframe
final_cols = sel_imputated_cols + sel_non_imputated_cols

In [26]:
len(final_cols)

88

In [27]:
df_final = df[final_cols]

In [31]:
df_final

,bankacctype_710L,cardtype_51L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,eir_270L,inittransactioncode_186L,interestrate_311L,lastapprcredamount_781A,lastrejectcredamount_222A,...,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,previouscontdistrict_112M,sellerplacecnt_915L,sellerplacescnt_216L
0,NA,NOCARD,0.0,0.0,0.0,0.4500,CASH,0.4500,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,a55475b1,0.0,0.0
1,NA,NOCARD,0.0,0.0,0.0,0.2999,CASH,0.2999,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,a55475b1,0.0,0.0
2,NA,NOCARD,0.0,0.0,0.0,0.4500,CASH,0.4500,0.0,10000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,a55475b1,0.0,0.0
3,NA,NOCARD,0.0,0.0,0.0,0.4200,CASH,0.4200,0.0,59999.8,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,a55475b1,1.0,1.0
4,NA,NOCARD,0.0,0.0,0.0,0.4500,CASH,0.4500,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,a55475b1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,CA,NOCARD,15.0,24.0,0.0,0.4200,CASH,0.4200,20020.0,150000.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,P123_39_170,0.0,8.0
1526655,CA,NOCARD,17.0,21.0,0.0,0.4000,CASH,0.4000,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,P162_18_172,0.0,3.0
1526656,CA,NOCARD,0.0,7.0,0.0,0.4200,CASH,0.4200,3998.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,P133_44_167,0.0,1.0
1526657,CA,NOCARD,7.0,23.0,0.0,0.4200,CASH,0.4200,0.0,2198.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,P123_6_84,0.0,4.0


In [ ]:
df_final.describe()

In [59]:
def euclidean_distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

In [61]:
def knn_graph(df, k, verbose=False):
    points = [p[1:] for p in df.itertuples()]
    g = nx.Graph()
    iterpoints = tqdm(enumerate(points), total=len(points)) if verbose else enumerate(points)
    for i, p in iterpoints:
        distances = map(lambda x: euclidean_distance(p, x), points)
        closests = np.argsort(distances)[1:k+1] 
        for c in closests:
            g.add_edge(i, c, weight=distances[c])
        g.nodes[i]['pos'] = p
    return g